# SPARQL Queries to retrieve data from the constructed RDF graph


In [1]:
# Import necessary libraries
# !pip install rdflib # uncomment if necessary
import rdflib
from rdflib import Graph, URIRef, Namespace, SDO
import pandas as pd

# Load the graph (assuming it's saved as 'food_nutrition_graph.rdf')
g = Graph()

g.parse("KEN4256-integrated-KG-Team6.ttl", format="ttl")

prefix_string = """ PREFIX kgs: <http://kg-course.io/food-nutrition/schema/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <https://schema.org/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>"""

print(len(g))

33804


## Find recipes that are risky for individuals allergic to mango

This query retrieves recipes that contain mango as an ingredient, which could be risky for individuals with a mango allergy. It checks the recipeIngredient property for any mention of "mango" (case-insensitive) and returns the recipe name and ingredient list for those recipes.

In [2]:
results1 = g.query(
    f"""
    {prefix_string}
    SELECT * WHERE {{
        ?recipe a schema:Recipe ;
            schema:name ?name ;
            schema:recipeIngredient ?ingredientList .
        FILTER(CONTAINS(lcase(str(?ingredientList)), "mango"))
    }}
    LIMIT 20
    """
)
results1 = pd.DataFrame(results1.bindings)
results1

,ingredientList,name,recipe
0,"cumin seeds, red pepper flakes, turmeric, ging...",Chicken Curry,http://kg-course.io/food-nutrition/recipe/159
1,"serrano chili, mango, tamarind pulp, garlic, f...",Mango Salsa,http://kg-course.io/food-nutrition/recipe/340
2,"mango, bean sprouts, carrot, lemon juice, low ...",Green Mango Salad With Cilantro Vinaigrette,http://kg-course.io/food-nutrition/recipe/534
3,"mango, strawberries, nectarines, fresh raspber...",Fruit Salsa,http://kg-course.io/food-nutrition/recipe/596
4,"cumin seeds, fennel seeds, garam masala, mango...",Chaat Masala,http://kg-course.io/food-nutrition/recipe/99


## List all pies tagged as 'healthy' with a cooking time of less than 2 hours

This query retrieves recipes that are categorised as pies, tagged with the keyword "healthy", and have a cooking time of less than 120 minutes (2 hours). It checks the recipeCategory for "pie", the keywords for "healthy", and the cookTime for values less than 7200 seconds. The results include the recipe name, keywords, category, and cooking time, ordered by cooking time in descending order.

In [3]:
results2 = g.query(
    f"""
    {prefix_string}
    SELECT * WHERE {{
        ?recipe a schema:Recipe ;
            schema:name ?name ;
            schema:keywords ?keywords ;
            schema:recipeCategory ?recipeCategory ;
            schema:cookTime ?cookTime .

        FILTER(CONTAINS(lcase(str(?recipeCategory)), "pie") && CONTAINS(lcase(str(?keywords)), "healthy") && ?cookTime < "120"^^xsd:decimal)
    }}
    ORDER BY DESC(?cookTime)
    LIMIT 20
    """
)
results2 = pd.DataFrame(results2.bindings)
results2

,cookTime,keywords,name,recipe,recipeCategory
0,50.0,"Dessert, Healthy, Weeknight, Oven, < 4 Hours",Buttermilk Pie With Gingersnap Crumb Crust,http://kg-course.io/food-nutrition/recipe/45,Pie
1,50.0,"Dessert, Healthy, Weeknight, Oven, < 4 Hours",Grape-Nuts Pie,http://kg-course.io/food-nutrition/recipe/563,Pie
2,45.0,"Dessert, Berries, Fruit, Low Protein, Low Chol...",Blue Mountain Blueberry Pie,http://kg-course.io/food-nutrition/recipe/432,Pie
3,25.0,"Dessert, Apple, Fruit, Low Protein, Low Choles...",Ritz Mock Apple Pie IIi,http://kg-course.io/food-nutrition/recipe/913,Pie
4,15.0,"Dessert, Fruit, Low Protein, Healthy, < 60 Min...",Raisin Cream Pie,http://kg-course.io/food-nutrition/recipe/1045,Pie
5,15.0,"Dessert, Rice, Healthy, < 30 Mins, Oven, Easy",Rice Case for Quiches or Pies (Gluten-Free),http://kg-course.io/food-nutrition/recipe/1059,Pie
6,11.0,"Dessert, Low Protein, Healthy, < 30 Mins, Oven",Vinegar Pie,http://kg-course.io/food-nutrition/recipe/367,Pie
7,0.0,"Dessert, Apple, Fruit, Vegetable, Low Protein,...",Pumpkin Apple Pie,http://kg-course.io/food-nutrition/recipe/1032,Pie


## List all restaurants in New Delhi that serve Chinese cuisine and offer online delivery

In [4]:
results3 = g.query(
    f"""
    {prefix_string}
    SELECT * WHERE {{
        ?restaurant a schema:Restaurant ;
            schema:name ?name ;
            schema:addressLocality ?locality ;
            schema:servesCuisine ?servesCuisine ;
            kgs:hasOnlineDelivery ?deliversOnline .

        FILTER(CONTAINS(lcase(str(?locality)), "new delhi") && CONTAINS(lcase(str(?servesCuisine)), "chinese") && ?deliversOnline = "true"^^xsd:boolean)
    }}
    ORDER BY ASC(?name)
    LIMIT 10
    """
)
results3 = pd.DataFrame(results3.bindings)
results3

,deliversOnline,locality,name,restaurant,servesCuisine
0,true,"Pitampura, New Delhi",361 Restaurant & Banquet,http://kg-course.io/food-nutrition/restaurant/...,"North Indian, Chinese, Seafood"
1,true,"Aggarwal City Mall, Pitampura, New Delhi",4 on 44 Restaurant & Bar,http://kg-course.io/food-nutrition/restaurant/...,"North Indian, Chinese"
2,true,"Vasant Vihar, New Delhi",AOWLS,http://kg-course.io/food-nutrition/restaurant/...,"Chinese, Fast Food"
3,true,"Laxmi Nagar, New Delhi",Aashirwad Restaurant,http://kg-course.io/food-nutrition/restaurant/...,"North Indian, Chinese"
4,true,"Kirti Nagar, New Delhi",Above & Beyond,http://kg-course.io/food-nutrition/restaurant/...,"North Indian, Chinese, Mughlai, Continental"
5,true,"R K Puram, New Delhi","Ada ""e"" Haandi's",http://kg-course.io/food-nutrition/restaurant/...,"North Indian, Mughlai, Chinese"
6,true,"Moti Bagh, New Delhi",Ada e Handi Restaurant,http://kg-course.io/food-nutrition/restaurant/...,"Mughlai, North Indian, Chinese"
7,true,"Rajinder Nagar, New Delhi",Ada'e Handi,http://kg-course.io/food-nutrition/restaurant/...,"North Indian, Mughlai, Chinese"
8,true,"Green Park, New Delhi",Adyar Ananda Bhavan,http://kg-course.io/food-nutrition/restaurant/769,"South Indian, North Indian, Street Food, Chine..."
9,true,"Lajpat Nagar 2, New Delhi",Aggarwal Sweet Corner,http://kg-course.io/food-nutrition/restaurant/...,"North Indian, Chinese, Street Food, Mithai"


## Find the average cost of two dining at restaurants in Davenport that serve Asian food

In [5]:
results4 = g.query(
    f"""
    {prefix_string}
    SELECT (AVG(?cost) as ?avgCostof)  WHERE {{
        ?restaurant a schema:Restaurant ;
            schema:name ?name ;
            schema:addressLocality ?locality ;
            schema:servesCuisine ?servesCuisine ;
            kgs:averageCostOfTwo ?cost .

        FILTER(CONTAINS(lcase(str(?locality)), "davenport") && CONTAINS(lcase(str(?servesCuisine)), "asian"))
    }}
    ORDER BY DESC(?cost)
    """
)
results4 = pd.DataFrame(results4.bindings)
results4

,avgCostof
0,0


## Recommend the top 5 desserts published after 2000 that are labeled as ‘Easy’ and low in calories(< 300), along with their images

In [6]:
results5 = g.query(
    f"""
    {prefix_string}
    SELECT ?recipe ?imageURL ?name ?keywords ?datePublished ?calories WHERE {{
        ?recipe a schema:Recipe ;
            schema:name ?name ;
            schema:keywords ?keywords ;
            schema:datePublished ?datePublished ;
            schema:image ?imageURL ;
            kgs:hasNutrition ?nutrition .
        ?nutrition a schema:NutritionInformation ;
            schema:calories ?calories .

        FILTER(CONTAINS(lcase(str(?keywords)), "easy") && ?calories < 300 && ?datePublished > "2000-01-01T00:00:00Z"^^xsd:dateTime)
    }}
    ORDER BY DESC(?calories)
    LIMIT 5
    """
)

results5 = pd.DataFrame(results5.bindings)
results5

""


## Identify the top 10 highly rated beverages, including their preparation time and sugar content

In [ ]:
results6 = g.query(
    f"""
    {prefix_string}
    SELECT * WHERE {{
        ?recipe a schema:Recipe ;
            schema:name ?name ;
            schema:keywords ?keywords ;
            schema:recipeCategory ?recipeCategory ;
            schema:prepTime ?prepTime ;
            kgs:hasNutrition ?nutrition ;
            kgs:hasReview ?review .

        ?nutrition a schema:NutritionInformation ;
            schema:sugarContent ?sugar .

        ?review a schema:Review ;
            kgs:hasSentiment ?sentiment .

        FILTER(CONTAINS(lcase(str(?keywords)), "beverage") && CONTAINS(lcase(str(?recipeCategory)), "beverage"))
    }}
    ORDER BY DESC(?sentiment)
    LIMIT 10
    """
)
results6 = pd.DataFrame(results6.bindings)
results6

## Identify the highest-rated recipes containing protein-rich (> 20) ingredients and check if their corresponding cuisines are commonly available in the USA restaurants

In [ ]:
results7 = g.query(
    f"""
    {prefix_string}
    SELECT * WHERE {{
        ?recipe a schema:Recipe ;
            schema:name ?name ;
            schema:recipeCuisine ?recipeCuisine ;
            kgs:hasReview ?review ;
            kgs:hasNutrition ?nutrition .

        ?review a schema:Review ;
            kgs:hasSentiment ?sentiment .

        ?nutrition a schema:NutritionInformation ;
            schema:proteinContent ?protein .

        ?restaurant a schema:Restaurant ;
            schema:addressLocality ?locality ;
            schema:servesCuisine ?servesCuisine .

        FILTER( ?protein > 20 && CONTAINS(lcase(str(?locality)), "usa") && CONTAINS(lcase(str(?servesCuisine)), str(?recipeCuisine)))
    }}
    LIMIT 10
    """
)
results7 = pd.DataFrame(results7.bindings)
results7

## Find the top 5 healthiest recipes based on a nutrition density score (NDS), along with their average sentiment from reviews. Check if restaurants serve cuisines linked to these recipes and retrieve their aggregate restaurant ratings.

To calculate an NDS, you will incorporate weighting factors based on dietary importance, i.e., high protein, high fiber, low sugar.

> 𝑁𝐷𝑆 = (1.0 × 𝑃𝑟𝑜𝑡𝑒𝑖𝑛(𝑔)) + (1. 5 × 𝐹𝑖𝑏𝑒𝑟(𝑔)) − (2. 0 × 𝑆𝑢𝑔𝑎𝑟(𝑔))

Protein is essential for muscle and metabolism. Fiber is crucial for digestion and satiety. Excess sugar contributes to metabolic issues, hence higher penalty

In [ ]:
results8 = g.query(
    f"""
    {prefix_string}
    SELECT ?recipe ?name ?nds ?avgSentiment ?exampleRestaurant ?avgRestaurantRating WHERE {{
    ?recipe a schema:Recipe ;
            schema:name ?name ;
            schema:recipeCuisine ?recipeCuisine ;
            kgs:hasReview ?review ;
            kgs:hasNutrition ?nutrition .

    ?nutrition a schema:NutritionInformation ;
               schema:proteinContent ?protein ;
               schema:fiberContent ?fiber ;
               schema:sugarContent ?sugar .

    ?review a schema:Review ;
            kgs:hasSentiment ?sentiment .

    BIND(1.0 * ?protein + 1.5 * ?fiber - 2.0 * ?sugar AS ?nds)

    OPTIONAL {{
        ?restaurant a schema:Restaurant ;
                    schema:servesCuisine ?recipeCuisine ;
                    schema:name ?restaurantName ;
                    schema:aggregateRating ?rating .
    }}
    }}
    GROUP BY ?recipe ?name ?nds ?recipeCuisine
    ORDER BY DESC(?nds)
    LIMIT 5
    """
)
results8 = pd.DataFrame(results8.bindings)
results8